# 最適化問題（シフトスケジューリング問題）をPuLPで解く際のサンプルコード

## 参考: 
- [組合せ最適化でナーススケジューリング問題を解く](https://qiita.com/SaitoTsutomu/items/a33aba1a95828eb6bd3f)
- [「組合せ最適化でナーススケジューリング問題を解く」を自分なりに理解してみた](https://naomichi.work/2018/11/21/2018-11-21-175004/)

### メモ:
- 実はインターフェースは**PuLP**じゃなくて**Python-MIP**の方が同梱されている商用利用できるリゾルバが頭良くてこのほうがいいのでは？との意見が散見される。

### Step1: モジュールインポート

In [1]:
import numpy as np, pandas as pd
from pulp import *
from ortoolpy import addvars, addbinvars
from io import StringIO

### Step2: データ読み込みと整形

### 変数, 定数の意味対応一覧

- shift_h: 希望シフトのデータフレーム
- time_zone: 朝、昼, 夜などの時間帯
- require: 必要人数
- staff_N: スタッフN
- dow: 曜日（Day of weekの略)
- FRAME_TOTAL: コマの総数
- STAFF_TOTAL: 従業員総数
- STAFF_IDX_LIST: 従業員のインデックスリスト
- ADMIN_STAFF_IDX_LIST: 管理者権限従業員のインデックスリスト
- PW_N: 制約条件Nのペナルティweight
    - DIFF_REQUIRE: 必要人数差
    - CANNOT_ASSIGN: 希望不可シフトへのアサイン
    - MIN_ASSING: 最低コマ数（各従業員に対して、希望シフトの1/2）
    - LACK_ADMIN: 管理者不足
- assign_frame: 従業員数×コマ数の0-1変数
- is_require_diff: 必要人数差がある（必要人数がアサイン人数を下回る）かどうかのフラグ
- is_lack_admin: 管理者不足かどうかのフラグ
- min_assign: 最低コマ数を満たしているかどうかのフラグを書くのした配列


In [2]:
# 希望シフトのデータ
shift_sample = pd.read_table(StringIO("""\
曜日\t月\t月\t月\t火\t火\t火\t水\t水\t水\t木\t木\t木\t金\t金\t金\t土\t土\t土\t日\t日\t日
時間帯\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜
必要人数\t2\t3\t3\t2\t3\t3\t2\t3\t3\t1\t2\t2\t2\t3\t3\t2\t4\t4\t2\t4\t4
従業員0\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t
従業員1\t○\t○\t○\t\t\t\t○\t○\t○\t\t\t\t○\t○\t○\t\t\t\t\t\t
従業員2\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○
従業員3\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○
従業員4\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○
従業員5\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t\t\t\t\t\t
従業員6\t\t\t\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○\t○\t○\t○
従業員7\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t
従業員8\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○
従業員9\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○""")).T

In [3]:
# 確認
display(shift_sample)

,0,1,2,3,4,5,6,7,8,9,10,11
曜日,時間帯,必要人数,従業員0,従業員1,従業員2,従業員3,従業員4,従業員5,従業員6,従業員7,従業員8,従業員9
月,朝,2,○,○,NaN,○,NaN,○,NaN,NaN,NaN,NaN
月.1,昼,3,NaN,○,NaN,○,NaN,○,NaN,○,NaN,NaN
月.2,夜,3,NaN,○,NaN,○,○,○,NaN,NaN,○,NaN
火,朝,2,○,NaN,NaN,○,NaN,○,NaN,NaN,NaN,NaN
火.1,昼,3,NaN,NaN,NaN,○,NaN,○,NaN,○,NaN,NaN
火.2,夜,3,NaN,NaN,NaN,○,○,○,NaN,NaN,○,NaN
水,朝,2,○,○,NaN,○,NaN,○,NaN,NaN,NaN,NaN
水.1,昼,3,NaN,○,NaN,○,NaN,○,NaN,○,NaN,NaN
水.2,夜,3,NaN,○,NaN,○,○,○,NaN,NaN,○,NaN


In [4]:
shift_h = pd.read_csv('./data/2021-11-first.csv',  header=None)
shift_h = shift_h.set_index([0])

display(shift_h)

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,,,,,,,,,,,,,,,,,,,,,
曜日,時間帯,必要人数,根本修,目黒のどか,奥咲姫,岩村一樹,大井惇史,酒匂あかね,髙橋茉里那,衣笠夏大,...,齋藤明日美,畠山史哉,ガンボルド・ウヤンガ,佐藤陽輝,渡邉尊人,小山桃花,川﨑優希,築田麗,吉田太一,今村小雪
月,朝,2,NaN,○,NaN,○,NaN,NaN,NaN,NaN,...,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
月,昼,3,○,○,NaN,○,NaN,NaN,NaN,NaN,...,○,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN
月,夜,4,○,NaN,○,NaN,NaN,○,NaN,○,...,NaN,○,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN
火,朝,2,NaN,○,NaN,○,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
火,昼,3,NaN,○,NaN,○,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,○
火,夜,4,NaN,NaN,○,NaN,NaN,○,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
水,朝,2,NaN,○,NaN,○,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN
水,昼,3,NaN,○,NaN,○,NaN,NaN,NaN,○,...,NaN,NaN,NaN,NaN,NaN,○,NaN,○,NaN,NaN


In [5]:
# 0行目を削除してカラムを英語化してDataframeにつける
staff_names = shift_h.loc['曜日', 3:].tolist()
default_columns = ['time_zone', 'require']
default_columns.extend(staff_names)
shift_h.columns = (default_columns)
shift_h = shift_h.drop(index='曜日')
shift_h = shift_h.iloc[1:]

# 必要人数をint型に変換
shift_h.require = shift_h.require.astype(int)

# 希望シフトの有無をbooleanにビット変換
shift_h.iloc[:,2:] = ~shift_h.iloc[:,2:].isnull()

# 曜日を入れ込み1文字目を適用
shift_h.insert(0, 'dow', shift_h.index.str[0])

# インデックスリセット
shift_h.reset_index(drop=True, inplace=True)

# dow, require, time_zoneをカラムの後ろに移動
shift_h = shift_h.iloc[:,list(range(3,shift_h.shape[1]))+[0,1,2]]

In [6]:
# DFが意図通り作られているか確認
display(shift_h[:3])

,根本修,目黒のどか,奥咲姫,岩村一樹,大井惇史,酒匂あかね,髙橋茉里那,衣笠夏大,熊谷翔太郎,鈴木文萌,...,佐藤陽輝,渡邉尊人,小山桃花,川﨑優希,築田麗,吉田太一,今村小雪,dow,time_zone,require
0,True,True,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,月,昼,3
1,True,False,True,False,False,True,False,True,False,True,...,False,False,False,True,False,False,False,月,夜,4
2,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,火,朝,2


### Step3: モデルに用いる定数と変数を定義

In [7]:
# 必要な定数化
FRAME_TOTAL = shift_h.shape[0]
STAFF_TOTAL = shift_h.shape[1]-3
STAFF_IDX_LIST = list(range(STAFF_TOTAL))
# TODO: 個人名で特定できるように修正
ADMIN_STAFF_IDX_LIST = [0,1,2,3,4,5,6,7,8,14]

# 各種ペナルティweight    
PW_DIFF_REQUIRE = 10
PW_CANNOT_ASSIGN = 100
PW_MIN_ASSIGN = 10
PW_LACK_ADMIN = 100


# 従業員数×コマ数の0-1変数を格納した2次元配列の作成。これがアサイン用のシフト表になる。
assign_frame =  np.array(addbinvars(FRAME_TOTAL, STAFF_TOTAL))

# 必要人数差を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで差があるかないかのフラグとして利用)
shift_h['is_require_diff'] = addvars(FRAME_TOTAL)

#  最低コマ数を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで最低アサインを満たしているかどうかのフラグとして利用)
min_assign = addvars(STAFF_TOTAL)

#  管理者不足を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで管理者不足かどうかのフラグとして利用)
shift_h['is_lack_admin'] = addvars(FRAME_TOTAL)

### Step4: 定式化

In [8]:
# 数理モデル作成(最小化問題)
model = LpProblem()

# 目的関数定義
# 目的関数1: 必要人数が一致していない場合
function_1 = PW_DIFF_REQUIRE * lpSum(shift_h.is_require_diff)

# 目的関数2: 希望不可にアサインしてしまった時。制約も巻き込んでる?
# rがそのシフト時間帯の行
# shift_h.apply(r: 1-r[STAFF_IDX_LIST])がシフトrのスタッフNが希望しているかどうかをのフラグをまとめたもの、希望してない部分が1になる（1 - False = 1, 1 -True = 0）。
# assign_frame[r.name]がシフトrのアサイン情報
# shift_h.apply(lambda r: lpDot(1-r[STAFF_IDX_LIST], assign_frame[r.name]), 1)が希望してないところにシフトが割り当てられてしまっている部分の一覧
function_2 = PW_CANNOT_ASSIGN * lpSum(shift_h.apply(lambda r: lpDot(1-r[STAFF_IDX_LIST], assign_frame[r.name]), 1))

# 目的関数3: 最低コマ数を満たしていない時
function_3 = PW_MIN_ASSIGN * lpSum(min_assign)

# 目的関数4: 管理者不足が存在する時
function_4 = PW_LACK_ADMIN * lpSum(shift_h.is_lack_admin)

# 最終的な目的関数
model += function_1 + function_2 + function_3 + function_4

# 制約付与
# 制約1: 必要人数が一致していない場合制約をつける。プラマイそれぞれについて
# 制約2: 管理者がいない場合に制約つける
for _, r in shift_h.iterrows():
    model += r.is_require_diff >=  (lpSum(assign_frame[r.name]) - r.require)
    model += r.is_require_diff >= -(lpSum(assign_frame[r.name]) - r.require)
    model += lpSum(assign_frame[r.name, ADMIN_STAFF_IDX_LIST]) + r.is_lack_admin >= 1

# 制約3: 希望シフトが1/2以上アサインされていない場合制約を付与する
for j ,n in enumerate((shift_h.iloc[:,STAFF_IDX_LIST].sum(0)+1)//2):
    model += lpSum(assign_frame[:, j]) + min_assign[j] >= n

### Step5: 実行

In [9]:
# 実行
%time model.solve()

CPU times: user 74.5 ms, sys: 13.3 ms, total: 87.9 ms
Wall time: 269 ms


1

### Step6: 結果表示

In [10]:
# 結果表示

result = np.vectorize(value)(assign_frame).astype(int)
shift_h['result'] = [''.join(i*j for i, j in zip(r, shift_h.columns)) for r in result]
print('目的関数: ', value(model.objective))
print('---------')
print('結果')
display(shift_h[['dow','time_zone','result']])

目的関数:  0.0
---------
結果


,dow,time_zone,result
0,月,昼,目黒のどか山邉壮真佐藤陽輝
1,月,夜,奥咲姫鈴木文萌酒井翔大畠山史哉
2,火,朝,目黒のどか福田陽羽悠
3,火,昼,岩村一樹吉田太一今村小雪
4,火,夜,酒匂あかね熊谷翔太郎大谷海仁朝倉天音
5,水,朝,目黒のどか小山桃花
6,水,昼,山邉壮真飯田野乃築田麗
7,水,夜,根本修鈴木文萌酒井翔大松尾怜
8,木,朝,岩村一樹齋藤明日美
9,木,昼,岩村一樹齋藤明日美佐藤陽輝


実際のシフトとコスト面で比較してみる

実際のシフト: 

スタッフ全体を考えた時、配属されている総人数が実際のシフトと比較してどのくらいなのかを考える

シフトを組むスタッフが考えることが減る。
シフトを組むスタッフが考えることとして、
- このシフトで現場は回るのか？
- それぞれのスタッフが不満を抱かないか？
- 